In [2]:
#!/usr/bin/env python
import json
import os
from collections import defaultdict
#from emailtome import noticeEMail
import emailtome
import datetime
import sys
reload(sys)
sys.setdefaultencoding("ISO-8859-1")

In [8]:
class SetEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, set):
            return list(obj)
        return json.JSONEncoder.default(self, obj)

In [5]:
def get_user_data(event): #newest bio available from pushes, containning username,avatar url, gravatar_id, e-mail adress and real name
    user_data=defaultdict(dict)
    if 'payload' in event.keys():
        if 'shas' in event['payload'].keys() and len(event['payload']['shas']):
            if len(event['payload']['shas'][0])>=1:
                name=event['payload']['shas'][0][-1]
                e_mail=event['payload']['shas'][0][1]
        else:
            name='Nan'
            e_mail='Nan' 
    else:
            name='Nan'
            e_mail='Nan' 

    if 'actor' in event.keys(): 
        if 'id' in event['actor'].keys():
            user_data[event['actor']['id']]={'username':event['actor']['login'], 
                                         'avatar_url': event['actor']['avatar_url'],
                                         'gravatar_id': event['actor']['gravatar_id'],
                                         'e-mail_adress': e_mail,
                                         'name':name}
    return user_data

In [39]:
def write(p, file_name, dataset):
        with open(p+file_name+".json","w") as f:
            json.dump(dataset,f, cls=SetEncoder)
    

In [6]:
def read_aggr_file(p, directory, filename, id_repo): #aggregate the data and generates user based json files
    errors=[]
    with open(os.path.join(directory,filename), 'r') as myfile:
        data=myfile.read()
    #print 'opened the file'
    lines=data.splitlines()
    for i in range(0, len(lines)):
        try:
            event=json.loads(lines[i].encode('utf-8').strip())
            if 'repo' in event.keys():
                if 'id' in event['repo'].keys() and 'id' in event['actor'].keys():
                    id_repo[event['actor']['id']].add(event['repo'][u'id'])
                    user_id=event['actor']['id']
                    user_data.update(get_user_data(event))
        except ValueError as e:
            error=((str(filename)+','+str(e)+','+str(lines[i])+'\n'))
            with open(p+filename+"_errors.txt","a") as f:
                f.write(error)
    return id_repo

In [1]:
ls

2011-02.json                      githubarchive_2011-02-12.json
2011-03.json                      githubarchive_2011-02-13.json
2011-03users.json                 githubarchive_2011-02-14.json
2011-04.json                      githubarchive_2011-03-14.json
2011-04users.json                 githubarchive_2011-03-15.json
718497.txt                        githubarchive_2011-03-28.json
718505.txt                        githubarchive_2011-03-30.json
718510.txt                        githubarchive_2011-04-06.json
718512.txt                        githubarchive_2011-04-08.json
718513.txt                        githubarchivedownloader.py
718524.txt                        hst.txt
718534.txt                        new_code_for_aggregation.ipynb
Analyse_users.ipynb               new_code_for_aggregation.py
all_user_attributes.json          test.py
database_creation.ipynb           test.txt
database_creation.py              test2.py*
date_id_repos.json                test_new_code_for_aggregation.p

In [6]:
user_data=defaultdict(dict)
id_repo=defaultdict(set)

In [61]:
a=read_aggr_file('','', 'githubarchive_2011-03-15.json', id_repo)

In [59]:
cat githubarchive_2011-03-15.json_errors.txt

In [5]:
"""def get_activity(lines, aggr): #list of repositories pushed code by users by days
    id_repo=defaultdict(dict)

    for i in range(0, len(lines)):
        event=json.loads(lines[i].encode('utf-8').strip())
        #print event['type']
        if 'repo' in event.keys():
            if 'id' in event['repo'].keys() and 'id' in event['actor'].keys():
                id_repo[event['actor']['id']].add(event['repo'][u'id'])
    return id_repo"""

In [8]:
import os
sorted([i for i in os.listdir(os.getcwd()) if i.endswith(".json") and i.startswith("githubarchive")])[:5:2]

['githubarchive_2011-02-12.json',
 'githubarchive_2011-02-14.json',
 'githubarchive_2011-03-30.json']

In [7]:
import os
from collections import defaultdict
import json
directory=os.getcwd()
#old_date_s=os.listdir(os.getcwd())[0]
old_date_s='2011-02-12'
#path="/mnt/cns_storage/home/vorsi/derived_new"
#directory= '/mnt/cns_storage/home/github_shared/data/raw'
path=''
aggr='month'

id_repo=defaultdict(set)
user_data=defaultdict(dict)
count=0
files=sorted([i for i in os.listdir(directory) if i.endswith(".json") and i.startswith("githubarchive")])
number_of_elements=len(files)

for filename in files:
    count+=1
    date_s=filename.split('_')[1][:-len('.json')]
    print date_s, old_date_s, 'before openning the file'
    if aggr=='month':
        if date_s[:7]==old_date_s[:7]:
            id_repo=read_aggr_file(directory, filename, id_repo)
            old_date_s=date_s
            if count==number_of_elements:
                write(path, date_s[:7], id_repo)
                write(path, date_s[:7]+'users', user_data)
        else:
            if count==number_of_elements:
                print count
                write(path, date_s[:7], id_repo)
                write(path, date_s[:7]+'users', user_data)
            else:
                write(path, old_date_s[:7], id_repo)
                write(path, date_s[:7]+'users', user_data)
                id_repo=defaultdict(set)
                user_data=defaultdict(dict)
                id_repo=read_aggr_file(directory, filename, id_repo)
                old_date_s=date_s
                    
    if aggr=='year':
        if date_s[:4]==old_date_s[:4]:
            id_repo=read_aggr_file(directory, filename, id_repo)
            old_date_s=date_s
            if count==number_of_elements:
                write(path, date_s[:4], id_repo)
                write(path, date_s[:4]+'users', user_data)
        else:
            if count==number_of_elements:
                write(path, date_s[:4], id_repo)
                write(path, date_s[:4]+'users', user_data)
            else:
                write(path, old_date_s[:4], id_repo)
                write(path, date_s[:4]+'users', user_data)
                id_repo=defaultdict(set)
                id_repo=read_aggr_file(directory, filename, id_repo)
                old_date_s=date_s

2011-02-12 2011-02-12 before openning the file


TypeError: read_aggr_file() takes exactly 4 arguments (3 given)

In [ ]:
starttime=datetime.datetime.now()
usr='orsolya.vasarhelyi'
psw='Vlak201188'
fromaddr='orsolya.vasarhelyi@gmail.com'
toaddr='orsolya.vasarhelyi@gmail.com'
emailtome.noticeEMail(starttime, usr, psw, fromaddr, toaddr)

In [15]:
user_attributes={}
for filename in os.listdir(path):
    if filename.endswith("users.json"):
        with open(filename) as data_file:    
            data = json.load(data_file)
            user_attributes.update(data)
write(path, 'all_user_attributes', user_attributes)

In [18]:
starttime=datetime.datetime.now()
usr='orsolya.vasarhelyi'
psw='Vlak201188'
fromaddr='orsolya.vasarhelyi@gmail.com'
toaddr='orsolya.vasarhelyi@gmail.com'
emailtome.noticeEMail(starttime, usr, psw, fromaddr, toaddr)

In [21]:
#sorted(os.listdir(os.getcwd()))